# MONAI Rehberi 1.2(Classification)
### 1. MONAI ve Gerekli Kütüphanelerin Yüklenmesi

In [ ]:
!python -c "import monai" || pip install -q "monai-weekly[ignite, tqdm, tensorboard]"

- Bu komut, MONAI'nin yüklü olup olmadığını kontrol eder. Yüklü değilse, `pip install` komutu kullanılarak en güncel sürümü yüklenir.
- **`ignite`**, **`tqdm`** ve **`tensorboard`** gibi MONAI ile uyumlu ek kütüphaneler de yüklenir.

### 2. Gerekli Kütüphanelerin İçe Aktarılması

In [ ]:
import logging
import numpy as np
import os
from pathlib import Path
import sys
import tempfile
import torch
import ignite

from monai.apps import MedNISTDataset
from monai.config import print_config
from monai.data import DataLoader
from monai.engines import SupervisedTrainer, SupervisedEvaluator
from monai.handlers import (
    StatsHandler,
    TensorBoardStatsHandler,
    ValidationHandler,
    CheckpointSaver,
    CheckpointLoader,
    ClassificationSaver,
)
from monai.handlers.utils import from_engine
from monai.inferers import SimpleInferer
from monai.networks.nets import densenet121
from monai.transforms import LoadImageD, EnsureChannelFirstD, ScaleIntensityD, Compose, AsDiscreted

print_config()

- Yukarıdaki kod, MONAI'nin işleyişi için gerekli kütüphaneleri içe aktarır. 
- **`print_config()`**, MONAI'nin mevcut sürümünü ve sistem yapılandırmasını terminalde gösterir. Böylece sürüm uyumluluğu doğrulanır.

### 3. Veri Setinin Hazırlanması

In [ ]:
directory = os.environ.get("MONAI_DATA_DIRECTORY")
if directory is not None:
    os.makedirs(directory, exist_ok=True)
root_dir = tempfile.mkdtemp() if directory is None else directory
print(root_dir)

- `root_dir`, MedNIST veri setini depolamak için geçici bir dizin oluşturur. 
- Eğer ortam değişkeni olarak bir MONAI veri dizini belirtilmişse, bu kullanılır.

In [ ]:
transform = Compose(
    [
        LoadImageD(keys="image", image_only=True),
        EnsureChannelFirstD(keys="image"),
        ScaleIntensityD(keys="image"),
    ]
)

dataset = MedNISTDataset(root_dir=root_dir, transform=transform, section="training", download=True)
valdata = MedNISTDataset(root_dir=root_dir, transform=transform, section="validation", download=False)

- **`LoadImageD`**: Görüntüleri dosyadan yükler.
- **`EnsureChannelFirstD`**: Görüntülerin kanal boyutunu modele uygun şekilde düzenler. Örneğin, bir 2D görüntüyü `(Height, Width)` yerine `(Channels, Height, Width)` formatına çevirir.
- **`ScaleIntensityD`**: Görüntü yoğunluğunu normalize eder, yani tüm piksel değerlerini 0 ile 1 arasında ölçekler.
- **`MedNISTDataset`**: MONAI’nin sunduğu hazır bir tıbbi görüntü veri setidir. Eğitim ve doğrulama için veri bölümleri oluşturulur.

### 4. Model Tanımlama ve Eğitim

In [ ]:
max_epochs = 5
save_interval = 2
out_dir = "./eval"
model = densenet121(spatial_dims=2, in_channels=1, out_channels=6).to("cuda:0")

logging.basicConfig(stream=sys.stdout, level=logging.INFO)

evaluator = SupervisedEvaluator(
    device=torch.device("cuda:0"),
    val_data_loader=DataLoader(valdata, batch_size=512, shuffle=False, num_workers=4),
    network=model,
    inferer=SimpleInferer(),
    key_val_metric={"val_acc": ignite.metrics.Accuracy(from_engine(["pred", "label"]))},
    val_handlers=[StatsHandler(iteration_log=False), TensorBoardStatsHandler(iteration_log=False)],
)

trainer = SupervisedTrainer(
    device=torch.device("cuda:0"),
    max_epochs=max_epochs,
    train_data_loader=DataLoader(dataset, batch_size=512, shuffle=True, num_workers=4),
    network=model,
    optimizer=torch.optim.Adam(model.parameters(), lr=1e-5),
    loss_function=torch.nn.CrossEntropyLoss(),
    inferer=SimpleInferer(),
    train_handlers=[
        ValidationHandler(validator=evaluator, epoch_level=True, interval=1),
        CheckpointSaver(save_dir=out_dir, save_dict={"model": model}, save_interval=save_interval, save_final=True),
        StatsHandler(),
        TensorBoardStatsHandler(tag_name="train_loss", output_transform=from_engine(["loss"], first=True)),
    ],
)
trainer.run()

- **`densenet121`**: DenseNet-121 modelini oluşturur. Bu model, tıbbi görüntüler için optimize edilmiş bir sinir ağıdır.
  - `spatial_dims=2`: 2 boyutlu verilerle çalışır.
  - `in_channels=1`: Giriş kanalı sayısı (örneğin, gri tonlu görüntüler için 1).
  - `out_channels=6`: Çıkış sınıfı sayısıdır. (MedNIST veri setindeki sınıfların sayısı)
- **`SupervisedTrainer`**: Modeli eğitmek için kullanılan bir sınıftır.
- **`ValidationHandler`**: Her epoch sonunda doğrulama verisiyle performansı değerlendirir.
- **`CheckpointSaver`**: Modeli belirli aralıklarla kaydeder.


### Test ve Sonuçlar

In [ ]:
testdata = MedNISTDataset(root_dir=root_dir, transform=transform, section="test", download=False, runtime_cache=True)
evaluator.run()

- **`testdata`**: Test verisini yükler.
- **`evaluator.run()`**: Eğitilen modeli test verisi üzerinde çalıştırır ve doğruluk gibi metrikleri hesaplar.